This is created after checking EM for GMM, based on old versions. Old versions are put in 'old' folder.

In [1]:
from __future__ import division
from numpy import linalg as LA
from onebox import *
import matplotlib.pyplot as plt

In [4]:
discount = 0.99
nq = 10
nr = 2
na = 2

In [5]:
beta = 2 * 10**-6   # .001     # available food dropped back into box after button press
gamma = .3                     # reward becomes available
epsilon = .1                   # available food disappears
rho = 1 - 2 * 10**-6 #.99      # food in mouth is consumed
pushButtonCost = 0.6
parameters = [beta, gamma, epsilon, rho, pushButtonCost]

# Genearte data (softmax policy)

In [6]:
T = 1000
N = 10
hiddenInitial = 1
#np.random.randint(nq)
obsInitial = 0
print(N, "samples are taken. Each sample has", T, "actions and accordingly", T+1, "observations\n")


10 samples are taken. Each sample has 1000 actions and accordingly 1001 observations



In [7]:
# First generate the observations (softmax policy)

oneboxdata = oneboxMDPdata(discount, nq, nr, na, parameters, T, N)
oneboxdata.dataGenerate_sfm(hiddenInitial, obsInitial)   #softmax policy

belief = oneboxdata.belief
action = oneboxdata.action
reward = oneboxdata.reward
trueState = oneboxdata.trueState


# sampleNum * sampleTime * dim of observations(=2 here, action, reward)
# shape of obsN: N*T*2
obsN = np.array([action, reward])   # includes the action and the observable states
obsN = np.rot90(obsN, 1)[::-1]                 # transform to the shape sampleNum * 2 * sampleTime
obsN = np.transpose(obsN, [0, 2, 1])  
#obs = obsN[0, :, :]    # take the first sample of sequence

latN = belief.copy()

In [ ]:
# show data for optimal agent

data_true = np.reshape(oneboxdata.trueState, oneboxdata.sampleTime)
data_action = np.reshape(oneboxdata.action, oneboxdata.sampleTime)
data_reward = np.reshape(oneboxdata.reward, oneboxdata.sampleTime)
data_belief = np.reshape((oneboxdata.belief + 1 / 2) / float(nq), oneboxdata.sampleTime)
data_hybrid = np.reshape(oneboxdata.hybrid, oneboxdata.sampleTime)

actiontemp = data_action * (1 * (data_action % pb == 0)) / pb

time = np.arange(0, oneboxdata.sampleTime, 1)
if oneboxdata.sampleTime < 1000:
fig = plt.figure(figsize=(20, 8))
ax1 = fig.add_subplot(311)
ax3 = fig.add_subplot(312)
ax5 = fig.add_subplot(313)

ax1.plot(time, actiontemp, 'b*', label='action')
ax1.legend(loc=1)
ax1.set_xticks(np.arange(0, oneboxdata.sampleTime, 5))
ax1.set_xticks(np.arange(0, oneboxdata.sampleTime, 1), minor=True)
ax1.set_yticks(np.arange(0, 3, 1))
a = ax1.get_yticks().tolist()
a[0] = 'NA'
a[1] = 'pb'
ax1.set_yticklabels(a)
ax1.set_yticks(ax1.get_yticks()[0:2])
ax1.grid(which='minor', alpha=0.5)
ax1.grid(which='major', alpha=0.7)
ax1.set_ylabel('action')

ax2 = ax1.twinx()
ax2.plot(time, data_reward, 'g', label='reward')
ax2.legend(loc=1)
ax2.set_xticks(np.arange(0, oneboxdata.sampleTime, 5))
ax2.set_xticks(np.arange(0, oneboxdata.sampleTime, 1), minor=True)
ax2.set_yticks(np.arange(0, 3, 1))
ax2.grid(which='minor', alpha=0.5)
ax2.grid(which='major', alpha=0.7)

ax3.plot(time, data_true, 'c', label='trueState')
ax3.legend(loc=1)
ax3.set_xticks(np.arange(0, oneboxdata.sampleTime, 5))
ax3.set_xticks(np.arange(0, oneboxdata.sampleTime, 1), minor=True)
ax3.set_yticks(np.arange(0, 1.2))
ax3.grid(which='minor', alpha=0.5)
ax3.grid(which='major', alpha=0.7)

ax5.plot(time, data_belief, 'm', label='belief')
ax5.legend(loc=1)
ax5.set_xticks(np.arange(0, oneboxdata.sampleTime, 5))
ax5.set_xticks(np.arange(0, oneboxdata.sampleTime, 1), minor=True)
ax5.set_yticks(np.arange(0, 1, 2 * 1 / float(nq)))
ax5.set_yticks(np.arange(0, 1, 1 / float(nq)), minor=True)
ax5.grid(which='minor', alpha=0.5)
ax5.grid(which='major', alpha=0.7)
ax5.set_ylabel('belief')

plt.show()

In [ ]:
# write a high-dimensioanl data into a file

with file('latN_new.txt', 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(belief.shape))

    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in belief:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%-7.0f')

        # Writing out a break to indicate different slices...
        outfile.write('# New slice\n')

In [ ]:
# write a high-dimensioanl data into a file

with file('obsN_new.txt', 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(obsN.shape))

    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in obsN:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%-7.0f')

        # Writing out a break to indicate different slices...
        outfile.write('# New slice\n')

In [ ]:
# Read the array from disk

obs_data = np.loadtxt('obsN.txt')
# Note that this returned a 2D array!
#print new_data.shape

# However, going back to 3D is easy if we know the 
# original shape of the array
obs_data = obs_data.reshape((N, T, 2))

# Just to check that they're the same...
#np.all(obs_data == obsN)

In [ ]:
# Read the array from disk

lat_data = np.loadtxt('latN.txt')

# Note that this returned a 2D array!
#print new_data.shape

# However, going back to 3D is easy if we know the 
# original shape of the array
lat_data = lat_data.reshape((N, T))


In [8]:
# Take only one sample at this time 
obsN = obs_data.astype(int)
obs = obsN[0, :, :]

latN = lat_data.astype(int)
lat = latN[0]

NameError: name 'obs_data' is not defined

## Plot the Q contour (gamma and epsilon)

In [9]:
from matplotlib import cm

In [ ]:
# Compute likelihood with ground truth latent

lat = latN[0]
obs = obsN[0, :, :] 


beta_ini = 2 * 10**-6
rho_ini = 1 - 2 * 10**-6
pushButtonCost_ini = 0.6

step1 = 0.02
step2 = 0.02
N1 = 50
N2 = 50

gammaValue = np.zeros(N1)
epsilonValue = np.zeros(N2)
Qaux1 = np.zeros((N2, N1))    # Likelihood with ground truth latent
Qaux2 = np.zeros((N2, N1))    # Expected complete data likelihood
Qaux3 = np.zeros((N2, N1))    # Entropy of latent posterior



for i in range(N1):
    gammaValue[i] = step1 * (i + 1)    
    for j in range(N2):
        epsilonValue[j] = step2 * (j + 1)   
        para = [beta_ini, gammaValue[i], epsilonValue[j], rho_ini, 0.6]
        
        onebox = oneboxMDP(discount, nq, nr, na, para)
        onebox.setupMDP()
        onebox.solveMDP_sfm()
        ThA = onebox.ThA
        policy = onebox.softpolicy
        pi = np.ones(nq)/nq  # initialize the estimation of the belief state
        oneboxHMM = HMMonebox(ThA, policy, pi)
        
        Qaux1[j, i] = oneboxHMM.likelihood(lat, obs, ThA, policy)
        Qaux2[j, i] = oneboxHMM.computeQaux(obs, ThA, policy) 
        Qaux3[j, i] = oneboxHMM.latent_entr(obs) 



In [ ]:
Likelihood = Qaux2 + Qaux3

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
gammaValuemesh, epsilonValuemesh = np.meshgrid(gammaValue, epsilonValue)
cs3 = plt.contour(gammaValuemesh, epsilonValuemesh, Likelihood, 
                  np.arange(np.min(Likelihood), np.max(Likelihood), 5), cmap = cm.jet)
plt.xticks(np.arange(0, 1, 0.1))
plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Likelihood of observed data')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs3, inline=1, fontsize=10)
plt.colorbar()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
gammaValuemesh, epsilonValuemesh = np.meshgrid(gammaValue, epsilonValue)
cs3 = plt.contour(gammaValuemesh, epsilonValuemesh, Likelihood, 
                  np.arange(np.min(Likelihood), np.max(Likelihood), 5), cmap = cm.jet)
plt.xticks(np.arange(0, 1, 0.1))
plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Likelihood of observed data')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs3, inline=1, fontsize=10)
plt.colorbar()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
gammaValue1mesh, epsilonValue1mesh = np.meshgrid(gammaValue, epsilonValue)
cs1 = plt.contour(gammaValue1mesh, epsilonValue1mesh, Qaux1, 
                  np.arange(np.min(Qaux1), np.max(Qaux1), 10), cmap = cm.jet)
plt.xticks(np.arange(0, 1, 0.1))
plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Likelihood of observed with ground truth latent')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs1, inline=1, fontsize=10)
plt.colorbar()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
gammaValue2mesh, epsilonValue2mesh = np.meshgrid(gammaValue, epsilonValue)
cs = plt.contour(gammaValue2mesh, epsilonValue2mesh, Qaux2, 
                 np.arange(np.min(Qaux2), np.max(Qaux2), 5), cmap = cm.jet)
plt.xticks(np.arange(0, 1, 0.1))
plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Expected complete data likelihood')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs, inline=1, fontsize=10)
plt.colorbar()
plt.show()

In [ ]:
# Entropy of posterior
fig, ax = plt.subplots(figsize=(12, 9))
gammaValue3mesh, epsilonValue3mesh = np.meshgrid(gammaValue, epsilonValue)
cs1 = plt.contour(gammaValue3mesh, epsilonValue3mesh, Qaux3, 
                  np.arange(np.min(Qaux3), np.max(Qaux3), 5), cmap = cm.jet)
plt.xticks(np.arange(0, 1, 0.1))
plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Entropy of posterior')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs1, inline=1, fontsize=10)
plt.colorbar()
plt.show()

## EM (EG) algorithm

In [17]:
beta_ini = 2 * 10**-6
gamma_ini = 0.45
epsilon_ini = 0.3
rho_ini = 1 - 2 * 10**-6
pushButtonCost_ini = 0.6
    
parameters_ini = [beta_ini, gamma_ini, epsilon_ini, rho_ini, pushButtonCost_ini]
parameters_old = parameters_ini
    
print("The true paramters are:", parameters)
print("The initial estimation parameters are", parameters_old)
print("Now starting with some initialized value of the parameters, we are going to use EM(G) " \
"algorithm for parameter estimation:")

The true paramters are: [2e-06, 0.3, 0.1, 0.999998, 0.6]
The initial estimation parameters are [2e-06, 0.45, 0.3, 0.999998, 0.6]
Now starting with some initialized value of the parameters, we are going to use EM(G) algorithm for parameter estimation:


In [ ]:
NN1 = 100   # iteration number for the EM algorithm
eps = 10 ** -6

para_old_traj = []
para_new_traj = []

log_likelihoods_old = []
log_likelihoods_new = []
log_likelihoods_com_old = []    # old posterior, old parameters
log_likelihoods_com_new = []    # old posterior, new parameters
latent_entropies = []

count_E = 0
while count_E < NN1:
    
    print ('\nThe', count_E, '-th iteration of the EM(G) algorithm')       

    if count_E == 0:
        parameters_old = [beta_ini, gamma_ini, epsilon_ini, rho_ini, pushButtonCost_ini]
    else:
        parameters_old = np.copy(parameters_new)   # update parameters
    
    para_old_traj.append(np.array([parameters_old[1], parameters_old[2]]))  
    print (parameters_old)
    
    ##########  E-step ##########
    
    ## Use old parameters to estimate posterior
    oneboxGra = oneboxMDPder(discount, nq, nr, na, parameters_old)
    ThA_old = oneboxGra.ThA
    softpolicy_old = oneboxGra.softpolicy
    pi = np.ones(nq) / nq 
    oneHMM = HMMonebox(ThA_old, softpolicy_old, pi)
    
    ## Calculate likelihood of observed and complete date, and entropy of the latent sequence
    complete_likelihood_old = oneHMM.computeQaux(obs, ThA_old, softpolicy_old) 
    latent_entropy = oneHMM.latent_entr(obs) 
    log_likelihood = complete_likelihood_old + latent_entropy
    
    log_likelihoods_com_old.append(complete_likelihood_old)
    latent_entropies.append(latent_entropy)
    log_likelihoods_old.append(log_likelihood)
    
    print (complete_likelihood_old)
    print (log_likelihood)
    
    ## Check convergence
    if len(log_likelihoods_old) >= 2 and np.abs(log_likelihood - log_likelihoods_old[-2]) < eps:
        print ('EM has converged!')
        break
    
    
    ##########  M(G)-step ##########
    
    count_M = 0
    para_new_traj.append([])
    log_likelihoods_com_new.append([])
    log_likelihoods_new.append([])
    stepsize = 5 * 10 ** -6
    
    # Start the gradient descent from the old parameters
    parameters_new = np.copy(parameters_old)
    complete_likelihood_new = complete_likelihood_old
    likelihood = complete_likelihood_new + latent_entropy
    
    print ('\n M-step ')
    print (parameters_new)
    print (complete_likelihood_new)
    print (likelihood)
    
    while True:
        para_new_traj[count_E].append(np.array([parameters_new[1], parameters_new[2]]))
        log_likelihoods_com_new[count_E].append(complete_likelihood_new)
        log_likelihoods_new[count_E].append(likelihood)
        #print log_likelihoods_new
        
        ## Go the potential next point with gradient descent
        para_temp = parameters_new + stepsize * np.array(oneboxGra.dQauxdpara_sim(obs, parameters_new))
        temp = np.copy(para_temp)
        para_temp = np.copy(parameters_old)
        para_temp[1] = temp[1]
        para_temp[2] = temp[2]
        
        ## Check the ECDLL (old posterior, new parameters)
        onebox_new = oneboxMDP(discount, nq, nr, na, para_temp)
        onebox_new.setupMDP()
        onebox_new.solveMDP_sfm()
        ThA_new = onebox_new.ThA
        softpolicy_new = onebox_new.softpolicy
        complete_likelihood_new_temp = oneHMM.computeQaux(obs, ThA_new, softpolicy_new)
        
        ## Update the parameter if the ECDLL can be improved
        if complete_likelihood_new_temp > complete_likelihood_new:
            parameters_new = np.copy(para_temp)
            complete_likelihood_new = complete_likelihood_new_temp
            likelihood = complete_likelihood_new + latent_entropy
            
            print (parameters_new)
            print (complete_likelihood_new)
            print (likelihood)
        
            count_M += 1
        else:
            break
              
    count_E += 1



The 0 -th iteration of the EM(G) algorithm
[2e-06, 0.45, 0.3, 0.999998, 0.6]
-1750.8009373978502
-638.6544211995945

 M-step 
[2.00000e-06 4.50000e-01 3.00000e-01 9.99998e-01 6.00000e-01]
-1750.8009373978502
-638.6544211995945
[2.00000000e-06 4.45997143e-01 3.01305423e-01 9.99998000e-01
 6.00000000e-01]
-1747.6559338534607
-635.509417655205
[2.00000000e-06 4.42864002e-01 3.02214137e-01 9.99998000e-01
 6.00000000e-01]
-1745.7644439690528
-633.617927770797
[2.00000000e-06 4.40398317e-01 3.02819117e-01 9.99998000e-01
 6.00000000e-01]
-1744.6162144526634
-632.4696982544076
[2.00000000e-06 4.38446664e-01 3.03192613e-01 9.99998000e-01
 6.00000000e-01]
-1743.9106644738026
-631.7641482755469
[2.00000000e-06 4.36892108e-01 3.03390637e-01 9.99998000e-01
 6.00000000e-01]
-1743.4699897552632
-631.3234735570074
[2.00000000e-06 4.35645194e-01 3.03456518e-01 9.99998000e-01
 6.00000000e-01]
-1743.188652669113
-631.0421364708573
[2.00000000e-06 4.34637287e-01 3.03423701e-01 9.99998000e-01
 6.00000000e

[2.00000000e-06 4.27018877e-01 2.95310717e-01 9.99998000e-01
 6.00000000e-01]
-1742.258856536835
-630.1123403385793
[2.00000000e-06 4.27002775e-01 2.95279523e-01 9.99998000e-01
 6.00000000e-01]
-1742.2586155159495
-630.1120993176937
[2.00000000e-06 4.26987315e-01 2.95249570e-01 9.99998000e-01
 6.00000000e-01]
-1742.2583933107917
-630.1118771125359
[2.00000000e-06 4.26972470e-01 2.95220808e-01 9.99998000e-01
 6.00000000e-01]
-1742.2581884503563
-630.1116722521006
[2.00000000e-06 4.26958216e-01 2.95193190e-01 9.99998000e-01
 6.00000000e-01]
-1742.2579995791475
-630.1114833808917
[2.0000000e-06 4.2694453e-01 2.9516667e-01 9.9999800e-01 6.0000000e-01]
-1742.2578254476348
-630.111309249379
[2.00000000e-06 4.26931388e-01 2.95141203e-01 9.99998000e-01
 6.00000000e-01]
-1742.257664904203
-630.1111487059472
[2.00000000e-06 4.26918769e-01 2.95116749e-01 9.99998000e-01
 6.00000000e-01]
-1742.2575168873607
-630.1110006891049
[2.00000000e-06 4.26906652e-01 2.95093267e-01 9.99998000e-01
 6.00000000e

[2.00000000e-06 4.26636451e-01 2.94569354e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557774829306
-630.1092612846749
[2.00000000e-06 4.26635543e-01 2.94567592e-01 9.99998000e-01
 6.00000000e-01]
-1742.255776742875
-630.1092605446192
[2.00000000e-06 4.26634671e-01 2.94565900e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557760614459
-630.1092598631901
[2.00000000e-06 4.26633833e-01 2.94564275e-01 9.99998000e-01
 6.00000000e-01]
-1742.25577543406
-630.1092592358043
[2.00000000e-06 4.26633028e-01 2.94562714e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557748564643
-630.1092586582085
[2.00000000e-06 4.26632256e-01 2.94561215e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557743247596
-630.1092581265038
[2.00000000e-06 4.26631514e-01 2.94559775e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557738353337
-630.1092576370779
[2.00000000e-06 4.26630801e-01 2.94558393e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557733848666
-630.1092571866109
[2.00000000e-06 4.26630117e-01 2.94557065e-01 9.99998000e-01
 6.000

[2.00000000e-06 4.26614826e-01 2.94527401e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557683180607
-630.109252119805
[2.00000000e-06 4.26614775e-01 2.94527301e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557683173015
-630.1092521190458
[2.00000000e-06 4.26614725e-01 2.94527206e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557683166692
-630.1092521184135
[2.00000000e-06 4.26614678e-01 2.94527113e-01 9.99998000e-01
 6.00000000e-01]
-1742.255768316144
-630.1092521178882
[2.00000000e-06 4.26614632e-01 2.94527025e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557683157256
-630.1092521174699
[2.00000000e-06 4.26614588e-01 2.94526940e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557683153918
-630.1092521171361
[2.00000000e-06 4.26614546e-01 2.94526858e-01 9.99998000e-01
 6.00000000e-01]
-1742.255768315144
-630.1092521168882
[2.00000000e-06 4.26614506e-01 2.94526780e-01 9.99998000e-01
 6.00000000e-01]
-1742.2557683149673
-630.1092521167116
[2.00000000e-06 4.26614467e-01 2.94526705e-01 9.99998000e-01
 6.000

[2.00000000e-06 4.07683915e-01 2.86223075e-01 9.99998000e-01
 6.00000000e-01]
-1733.2151109698057
-618.3614098851492
[2.00000000e-06 4.07653588e-01 2.86162301e-01 9.99998000e-01
 6.00000000e-01]
-1733.2142066717552
-618.3605055870987
[2.00000000e-06 4.07624403e-01 2.86103808e-01 9.99998000e-01
 6.00000000e-01]
-1733.2133690280018
-618.3596679433454
[2.00000000e-06 4.07596316e-01 2.86047509e-01 9.99998000e-01
 6.00000000e-01]
-1733.2125931087667
-618.3588920241102
[2.00000000e-06 4.07569284e-01 2.85993321e-01 9.99998000e-01
 6.00000000e-01]
-1733.211874348814
-618.3581732641576
[2.00000000e-06 4.07543269e-01 2.85941165e-01 9.99998000e-01
 6.00000000e-01]
-1733.2112085245142
-618.3575074398577
[2.00000000e-06 4.07518231e-01 2.85890963e-01 9.99998000e-01
 6.00000000e-01]
-1733.2105917249337
-618.3568906402772
[2.00000000e-06 4.07494134e-01 2.85842642e-01 9.99998000e-01
 6.00000000e-01]
-1733.2100203297887
-618.3563192451322
[2.00000000e-06 4.07470942e-01 2.85796131e-01 9.99998000e-01
 6.0

...

[2.00000000e-06 3.12908344e-01 1.51200600e-01 9.99998000e-01
 6.00000000e-01]
-1745.0029430441027
-591.6505214404474
[2.00000000e-06 3.12906086e-01 1.51194890e-01 9.99998000e-01
 6.00000000e-01]
-1745.002935687915
-591.6505140842596
[2.00000000e-06 3.12903899e-01 1.51189360e-01 9.99998000e-01
 6.00000000e-01]
-1745.0029287905818
-591.6505071869265
[2.00000000e-06 3.12901782e-01 1.51184005e-01 9.99998000e-01
 6.00000000e-01]
-1745.0029223235117
-591.6505007198564
[2.00000000e-06 3.12899731e-01 1.51178818e-01 9.99998000e-01
 6.00000000e-01]
-1745.002916259945
-591.6504946562898
[2.00000000e-06 3.12897745e-01 1.51173795e-01 9.99998000e-01
 6.00000000e-01]
-1745.0029105747558
-591.6504889711005
[2.00000000e-06 3.12895821e-01 1.51168931e-01 9.99998000e-01
 6.00000000e-01]
-1745.0029052443906
-591.6504836407353
[2.00000000e-06 3.12893958e-01 1.51164220e-01 9.99998000e-01
 6.00000000e-01]
-1745.0029002467304
-591.6504786430751
[2.00000000e-06 3.12892154e-01 1.51159657e-01 9.99998000e-01
 6.00

[2.00000000e-06 3.12844120e-01 1.51038179e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028266516929
-591.6504050480376
[2.00000000e-06 3.12843888e-01 1.51037592e-01 9.99998000e-01
 6.00000000e-01]
-1745.002826580104
-591.6504049764487
[2.00000000e-06 3.12843664e-01 1.51037024e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028265131662
-591.6504049095108
[2.00000000e-06 3.12843446e-01 1.51036474e-01 9.99998000e-01
 6.00000000e-01]
-1745.002826450585
-591.6504048469296
[2.00000000e-06 3.12843235e-01 1.51035941e-01 9.99998000e-01
 6.00000000e-01]
-1745.002826392078
-591.6504047884227
[2.00000000e-06 3.12843031e-01 1.51035425e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028263373908
-591.6504047337355
[2.00000000e-06 3.12842834e-01 1.51034925e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028262862813
-591.650404682626
[2.00000000e-06 3.12842642e-01 1.51034441e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028262385197
-591.6504046348643
[2.00000000e-06 3.12842457e-01 1.51033973e-01 9.99998000e-01
 6.0000

[2.00000000e-06 3.12837521e-01 1.51021491e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028255950826
-591.6504039914273
[2.00000000e-06 3.12837498e-01 1.51021431e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028255949478
-591.6504039912925
[2.00000000e-06 3.12837475e-01 1.51021372e-01 9.99998000e-01
 6.00000000e-01]
-1745.002825594844
-591.6504039911886
[2.00000000e-06 3.12837452e-01 1.51021316e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028255947673
-591.650403991112
[2.00000000e-06 3.12837431e-01 1.51021261e-01 9.99998000e-01
 6.00000000e-01]
-1745.002825594714
-591.6504039910587
[2.00000000e-06 3.12837410e-01 1.51021208e-01 9.99998000e-01
 6.00000000e-01]
-1745.002825594683
-591.6504039910276
[2.00000000e-06 3.12837389e-01 1.51021157e-01 9.99998000e-01
 6.00000000e-01]
-1745.0028255946718
-591.6504039910164

The 15 -th iteration of the EM(G) algorithm
[2.00000000e-06 3.12837389e-01 1.51021157e-01 9.99998000e-01
 6.00000000e-01]
-1746.070802881152
-591.5993178055426

 M-step 
[2.00000000e-0

[2.00000000e-06 3.11854568e-01 1.48026935e-01 9.99998000e-01
 6.00000000e-01]
-1746.0289918547378
-591.5575067791283
[2.00000000e-06 3.11848813e-01 1.48012350e-01 9.99998000e-01
 6.00000000e-01]
-1746.0289436277462
-591.5574585521367
[2.00000000e-06 3.11843240e-01 1.47998226e-01 9.99998000e-01
 6.00000000e-01]
-1746.0288984083516
-591.557413332742
[2.00000000e-06 3.11837843e-01 1.47984548e-01 9.99998000e-01
 6.00000000e-01]
-1746.0288560090553
-591.5573709334458
[2.00000000e-06 3.11832617e-01 1.47971303e-01 9.99998000e-01
 6.00000000e-01]
-1746.028816254035
-591.5573311784256
[2.00000000e-06 3.11827556e-01 1.47958477e-01 9.99998000e-01
 6.00000000e-01]
-1746.028778978446
-591.5572939028366
[2.00000000e-06 3.11822655e-01 1.47946057e-01 9.99998000e-01
 6.00000000e-01]
-1746.0287440276795
-591.55725895207
[2.00000000e-06 3.11817910e-01 1.47934029e-01 9.99998000e-01
 6.00000000e-01]
-1746.028711256918
-591.5572261813086
[2.00000000e-06 3.11813314e-01 1.47922382e-01 9.99998000e-01
 6.000000

[2.00000000e-06 3.11691144e-01 1.47612762e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282271728506
-591.5567420972411
[2.00000000e-06 3.11690556e-01 1.47611272e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282266842848
-591.5567416086753
[2.00000000e-06 3.11689986e-01 1.47609828e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282262266337
-591.5567411510242
[2.00000000e-06 3.11689434e-01 1.47608430e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282257979666
-591.5567407223571
[2.00000000e-06 3.11688900e-01 1.47607076e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282253964567
-591.5567403208472
[2.00000000e-06 3.11688383e-01 1.47605765e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282250203943
-591.5567399447848
[2.00000000e-06 3.11687882e-01 1.47604495e-01 9.99998000e-01
 6.00000000e-01]
-1746.028224668186
-591.5567395925766
[2.00000000e-06 3.11687397e-01 1.47603265e-01 9.99998000e-01
 6.00000000e-01]
-1746.0282243383222
-591.5567392627127
[2.00000000e-06 3.11686927e-01 1.47602075e-01 9.99998000e-01
 6.0

In [ ]:
latent_entropies

In [ ]:
log_likelihoods_com_old

In [ ]:
log_likelihoods_old

In [ ]:
log_likelihoods_com_new

In [ ]:
log_likelihoods_new

## Plotting for EM

In [ ]:
#fig = plt.figure(figsize=(14, 6))

#fig.add_subplot(121)
fig, ax = plt.subplots(figsize=(12, 9))
gammaValuemesh, epsilonValuemesh = np.meshgrid(gammaValue, epsilonValue)
cs3 = plt.contour(gammaValuemesh, epsilonValuemesh, Likelihood, 
                  np.arange(np.min(Likelihood), np.max(Likelihood), 5))

plt.plot(np.vstack(para_old_traj)[:, 0], np.vstack(para_old_traj)[:, 1], marker='*', color = 'b')

plt.xticks(np.arange(0, 1, 0.1))
plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Contour of likelihood and trajectories')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs3, inline=1, fontsize=10)
plt.colorbar()

fig, ax = plt.subplots(figsize=(12, 9))
#fig.add_subplot(122)
#plt.plot(np.array([item for sublist in log_likelihoods_com_new for item in sublist]), color = 'b')
plt.plot(np.array([item for sublist in log_likelihoods_new for item in sublist]), color = 'r')
ax.grid()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
gammaValuemesh1, epsilonValuemesh1 = np.meshgrid(gammaValue[10:50], epsilonValue[10:40])
cs3 = plt.contour(gammaValuemesh1, epsilonValuemesh1, Likelihood[10:40, 10:50], 
                  np.arange(np.min(Likelihood[10:40, 10:50]), np.max(Likelihood[10:40, 10:50]), 5))

plt.plot(np.vstack(para_old_traj)[:, 0], np.vstack(para_old_traj)[:, 1], marker='*', color = 'b')

#plt.xticks(np.arange(0, 1, 0.1))
#plt.yticks(np.arange(0, 1, 0.1))
ax.grid()
ax.set_title('Contour of likelihood and trajectories')
plt.xlabel('gamma')
plt.ylabel('epsilon')
plt.clabel(cs3, inline=1, fontsize=10)
plt.colorbar()

plt.show()
